In [2]:
import sys 
sys.path.append("../")

import nerf_model
import dataloader

import plotly
import torch 
import cv2
from PIL import Image
import itertools 
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [33]:
from importlib import reload 
dataloader = reload(dataloader)

In [34]:
def visualize(coords, rgb): 
    a, b = coords.shape
    if b == 4:
        coords = coords.T
    d, N = coords.shape
    if type(rgb) != str:
        rgb = rgb.T
    plot_fig = go.Scatter3d(x=coords[0], y=coords[1], z=coords[2], 
    mode='markers', marker=dict(
       size=2,
       color=rgb
    ))
    return plot_fig

In [35]:
ds = dataloader.SyntheticDataset('../tests/test_data/', 'train', 4096, 50, 50)
batch = ds[0]
origin = batch['origin']
direc = batch['direc']
rgb = batch['rgba']

torch.Size([3, 50, 50])


In [31]:
print(rgb.shape)

torch.Size([3, 4096])


In [32]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

origin_fig = visualize(origin.T, rgb)
fig.add_trace(origin_fig)

direc_fig = visualize(direc.T, rgb)
fig.add_trace(direc_fig)


fig.show()

## Generate Test Image

In [ ]:
size = 50
gradient = np.expand_dims(np.linspace([0]*50, [1]*50, num=size, endpoint=True), -1)
image = np.zeros((size, size,3)) + gradient
image[:,:,0] = 1 - image[:,:,0] 
image = (image * 255).clip(0, 255).astype(np.uint8)
# image[:,:,3] = 1 - image[:,:,3] 
print(image.shape)

In [ ]:
im = Image.fromarray(image)
im.save('test1.png')

In [ ]:
def sample_random_coordinates(N, height, width): 
    """Returns [Nx4] randomly sampled coordinates in camera frame
    """
    xs = torch.randint(0, height, size=(N,))
    ys = torch.randint(0, width, size=(N,))
    return xs, ys

torch.manual_seed(0)
xs, ys = sample_random_coordinates(1, 50, 50)
print(xs, ys)
